In [ ]:
import os
import requests

class Tokenizer():
    def __init__(self):
        self.wordMap = {}
        self.wordMap2 = {}

    def format_text(self, t):
      t = t.replace(".", "").replace("?", "").replace("!", "")
      return t

    def build_vocab(self,file,fromWeb=False):
        self.wordMap = {}
        splitted = None
        if not fromWeb:
            with open(file, "r") as f:
                text = f.read()
                text = self.ormat_text
                splitted = text.split(' ')
        else:
            response = requests.get(file)
            response.raise_for_status()
            response.text = self.format_text(response.text)
            splitted = response.text.split(' ')

        for i in range(len(splitted)):
            if splitted[i].lower() not in self.wordMap:

              self.wordMap[splitted[i].lower()] = i
              self.wordMap2[i] = splitted[i].lower()

    def build_vocab_var(self, texts):
      combined_text = ""
      for j,text in enumerate(texts):
        combined_text += text

      combined_text = self.format_text(combined_text)
      splitted = combined_text.split(' ')
      for i in range(len(splitted)):
        if splitted[i].lower() not in self.wordMap:
          self.wordMap[splitted[i].lower()] = i
          self.wordMap2[i] = splitted[i].lower()


    def addToken(self, word):
        if word not in self.wordMap:
            self.wordMap[word] = len(self.wordMap)
            self.wordMap2[len(self.wordMap)] = word

    def encode(self, text):
        text = text.lower()
        text = self.format_text(text)

        encoded_list = []
        splitted_text = text.split()
        for token in splitted_text:
            encoded_list.append(self.wordMap[token])
        return encoded_list

    def decode(self, tokens):
        decoded_list = []
        for token in tokens:
            decoded_list.append(self.wordMap2[token])
        return decoded_list

    def test(self,url):
        response = requests.get(url)
        response.raise_for_status()
        print(response.text)

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("priyangshumukherjee/mental-health-text-classification-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'mental-health-text-classification-dataset' dataset.
Path to dataset files: /kaggle/input/mental-health-text-classification-dataset


In [2]:
import pandas as pd

data_csv = pd.read_csv(path + "/mental_health_combined_test.csv")
print(data_csv["text"][0])
print(data_csv["status"][0])

i don't understand whats wrong with me. i don't know why i freak out sometimes. like right now i'm just laying in bed nothing is happening but i can't keep the tears running from down my face. it's stupid. i'm honestly fine nothing is wrong. why on earth am i like this?
Anxiety


In [3]:
#label map
unique_labels = data_csv["status"].unique()
label_map = {}
for i, texts in enumerate(unique_labels):
  label_map[texts] = i
label_map

{'Anxiety': 0, 'Depression': 1, 'Normal': 2, 'Suicidal': 3}

In [ ]:
len(data_csv["text"])

992

In [4]:
texts = data_csv["text"]
status = data_csv["status"]

train_texts = texts[:700]
test_texts = texts[700:].reset_index(drop=True)
print(train_texts[0])
print(test_texts[0])

train_status = status[:700]
test_status = status[700:].reset_index(drop=True)
print(train_status[0])
print(test_status[0])

i don't understand whats wrong with me. i don't know why i freak out sometimes. like right now i'm just laying in bed nothing is happening but i can't keep the tears running from down my face. it's stupid. i'm honestly fine nothing is wrong. why on earth am i like this?
See ya dudes and dudettes... i am no longer part of the club :( officially 20 now... ill see yall in 93 years!
Anxiety
Normal


In [ ]:
tokenizer = Tokenizer()
tokenizer.wordMap.clear()
tokenizer.wordMap2.clear()
tokenizer.build_vocab_var(texts)

In [ ]:
tokenizer.wordMap["i"]
#tokenizer.wordMap2[1]

0

In [ ]:
len(tokenizer.wordMap)

13195

In [ ]:
enc = tokenizer.encode(train_texts[200])
dec = tokenizer.decode(enc)
t = ' '.join(dec)
t

"i just got hired for a job that pays pretty well, and the hours are well-suited for me, working about 4-6 hours per day, and i am super happy about it because i quit many jobs because of its long hours, and i hope this one last and that i wouldn't have any attacks at work"

In [7]:
#diffrent tokenizer
import tiktoken
tokenizer = tiktoken.get_encoding("o200k_base")
e = tokenizer.encode("<SOT>"  + train_texts[0] + " EOT")
print(e)
print(tokenizer.decode(e))

[30312, 2824, 29, 72, 4128, 4218, 55993, 8201, 483, 668, 13, 575, 4128, 1761, 4436, 575, 48851, 842, 10069, 13, 1299, 1849, 1954, 49232, 1327, 59127, 306, 3737, 6939, 382, 20230, 889, 575, 8535, 3357, 290, 37095, 6788, 591, 1917, 922, 4950, 13, 4275, 33883, 13, 49232, 39510, 8975, 6939, 382, 8201, 13, 4436, 402, 13440, 939, 575, 1299, 495, 30, 457, 2824]
<SOT>i don't understand whats wrong with me. i don't know why i freak out sometimes. like right now i'm just laying in bed nothing is happening but i can't keep the tears running from down my face. it's stupid. i'm honestly fine nothing is wrong. why on earth am i like this? EOT


In [80]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
  def __init__(self, tokenizer, texts, status):
    self.tokenizer = tokenizer
    self.texts = texts
    self.status = status

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = self.texts[idx]
    status = label_map[self.status[idx]]

    tokenized_text = self.tokenizer.encode(text)

    return torch.tensor(tokenized_text, dtype=torch.long), torch.tensor(status, dtype=torch.long)


In [81]:
#hyperparameters
if torch.cuda.is_available():
  device = "cuda"
else:
  device = "cpu"
batch_size = 8
embeded_dim = 100
hidden_dim = 128
output_dim = 4

In [10]:
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
  text_list, label_list = [], []
  for texts, labels in batch:
    text_list.append(texts)
    label_list.append(labels)

  text_padded = pad_sequence(text_list, batch_first=True, padding_value=0)
  labels = torch.tensor(label_list, dtype=torch.long)
  return text_padded, labels

train_set = TextDataset(tokenizer, train_texts, train_status)
test_set = TextDataset(tokenizer, test_texts, test_status)

train_loader = DataLoader(train_set,batch_size=batch_size,shuffle=True, collate_fn=collate_batch)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)
train_loader, test_loader

(<torch.utils.data.dataloader.DataLoader at 0x7eac7e6bb560>,
 <torch.utils.data.dataloader.DataLoader at 0x7eab92f9ccb0>)

In [ ]:
# Ersten Batch inspizieren
for texts, labels in train_loader:
    print("Text batch shape:", texts.shape)  # (batch_size, sequence_length)
    print("Label batch shape:", labels.shape)  # (batch_size,)
    print("Erste Text-IDs:", texts[0])
    print("Erstes Label:", labels[0])
    break

Text batch shape: torch.Size([8, 199])
Label batch shape: torch.Size([8])
Erste Text-IDs: tensor([ 30312,   2824,     29,     72,   5141,   4566,   5664,     13,    575,
          5141,  75680,    316,   2107,   3823,   5048,     13,    575,   5141,
          9791,    575,    673,   8847,     13,    575,   5141,   2447,    306,
          3047,     13,    575,   5141,   3778,   6687,    316,    290,  10668,
            13,    575,   5141,  24602,  78034,    869,    306,    290,   8709,
             0,    665,    481,   6423,    484,     30,   1354,    673,    261,
          1058,   1919,    575,  60424,    869,     11,    326,    575,    673,
          1299,    392, 173684,  37377,     11,    575,    717,    316,    413,
           668,    395,    261,   6062,   8415,  14202,     77,   2163,  18313,
           575,   5141,   3778,   6687,    316,  26585,    326,    922,  17715,
          7665,    328,    272,  47248,   1373,     13,    575,   5141,   4566,
           261,   5277,    575

In [ ]:
tokenizer.n_vocab

200019

In [ ]:
import torch.nn as nn

class TextModel(nn.Module):
  def __init__(self, vocab_size, embeded_dim, hidden_dim, output_dim):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embeded_dim)
    self.lstm = nn.LSTM(embeded_dim, hidden_dim, batch_first=True)
    self.fc = nn.Linear(hidden_dim, output_dim)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    embeded = self.embedding(x)
    lstm_out, _ = self.lstm(embeded)
    hidden_state = lstm_out[:, -1, :]  # Letzter Zeitschritt der Sequenz
    output = self.fc(hidden_state)
    return self.sigmoid(output)

model = TextModel(tokenizer.n_vocab , embeded_dim, hidden_dim, output_dim).to(device)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
epochs = 30

model.train()
for epoch in range(epochs):
  total_loss = 0

  for text, label in train_loader:
    text, label = text.to(device), label.to(device)

    outputs = model(text).squeeze(1)
    loss = loss_fn(outputs, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/30, Loss: 1.2383
Epoch 2/30, Loss: 1.2039
Epoch 3/30, Loss: 1.2022
Epoch 4/30, Loss: 1.2069
Epoch 5/30, Loss: 1.2049
Epoch 6/30, Loss: 1.2043
Epoch 7/30, Loss: 1.2020
Epoch 8/30, Loss: 1.2031
Epoch 9/30, Loss: 1.2015
Epoch 10/30, Loss: 1.2026
Epoch 11/30, Loss: 1.2026
Epoch 12/30, Loss: 1.2011
Epoch 13/30, Loss: 1.1697
Epoch 14/30, Loss: 1.1302
Epoch 15/30, Loss: 1.1163
Epoch 16/30, Loss: 1.0967
Epoch 17/30, Loss: 1.0881
Epoch 18/30, Loss: 1.2105
Epoch 19/30, Loss: 1.2141
Epoch 20/30, Loss: 1.2140
Epoch 21/30, Loss: 1.2136
Epoch 22/30, Loss: 1.2096
Epoch 23/30, Loss: 1.1764
Epoch 24/30, Loss: 1.0920
Epoch 25/30, Loss: 1.0188
Epoch 26/30, Loss: 0.9612
Epoch 27/30, Loss: 0.9591
Epoch 28/30, Loss: 0.9486
Epoch 29/30, Loss: 0.9687
Epoch 30/30, Loss: 0.9624


In [ ]:
idx = 3
text = tokenizer.encode(test_texts[idx])
input_tens = torch.tensor(text, dtype=torch.long).unsqueeze(0).to(device)
out = model(input_tens)
max_index_argmax = out.argmax()
print(max_index_argmax)
print(label_map[test_status[idx]])

tensor(2, device='cuda:0')
2


# Hier mal mit einem transformer

In [82]:
#more hyperparameters
EMBED_DIM = 256
NUM_ENCODER_LAYERS = 3
d_model = 4
NUM_HEADS = 4
dim_feedforward=2048

In [83]:
import math

class PositionalEncoding(nn.Module):
    """
    https://pytorch.org/tutorials/beginner/transformer_tutorial.html
    """

    def __init__(self, d_model, vocab_size=5000, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(vocab_size, d_model)
        position = torch.arange(0, vocab_size, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2).float()
            * (-math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1), :]
        return self.dropout(x)

In [96]:
class EncoderClassifier(nn.Module):
  def __init__(self,  vocab_size, embed_dim, num_layers, num_heads, output_size):
    super().__init__()
    self.emb = nn.Embedding(vocab_size, d_model)
    self.pos_encoder = PositionalEncoding(d_model=d_model, vocab_size=tokenizer.n_vocab)
    self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads,dim_feedforward=dim_feedforward ,batch_first=True)
    self.encoder = nn.TransformerEncoder(encoder_layer=self.encoder_layer, num_layers=num_layers)

    self.linear = nn.Linear(d_model, output_size)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    x = self.emb(x) * math.sqrt(d_model)
    x = self.pos_encoder(x)
    x = self.encoder(x)
    x = self.dropout(x)
    #x = x.max(dim=1)[0]
    #print(x)
    #x = x.mean(dim=1)
    x = x[:, 0, :]
    out = self.linear(x)

    return out

modelT = EncoderClassifier(tokenizer.n_vocab, embed_dim=EMBED_DIM, num_layers=NUM_ENCODER_LAYERS, num_heads=NUM_HEADS, output_size=4).to(device)
#modelT

In [97]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelT.parameters(), lr=1e-4 )

In [99]:
epochs = 30

modelT.train()
for epoch in range(epochs):
  total_loss = 0

  for text, label in train_loader:
    text, label = text.to(device), label.to(device)

    outputs = modelT(text)
    loss = loss_fn(outputs, label)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1/30, Loss: 1.0732
Epoch 2/30, Loss: 1.0731
Epoch 3/30, Loss: 1.0693
Epoch 4/30, Loss: 1.0540
Epoch 5/30, Loss: 1.0719
Epoch 6/30, Loss: 1.0716
Epoch 7/30, Loss: 1.0684
Epoch 8/30, Loss: 1.0497
Epoch 9/30, Loss: 1.0824
Epoch 10/30, Loss: 1.0637
Epoch 11/30, Loss: 1.0532
Epoch 12/30, Loss: 1.0362
Epoch 13/30, Loss: 1.0500
Epoch 14/30, Loss: 1.0422
Epoch 15/30, Loss: 1.0493
Epoch 16/30, Loss: 1.0354
Epoch 17/30, Loss: 1.0473
Epoch 18/30, Loss: 1.0580
Epoch 19/30, Loss: 1.0315
Epoch 20/30, Loss: 1.0372
Epoch 21/30, Loss: 1.0456
Epoch 22/30, Loss: 1.0267
Epoch 23/30, Loss: 1.0384
Epoch 24/30, Loss: 1.0208
Epoch 25/30, Loss: 1.0220
Epoch 26/30, Loss: 1.0126
Epoch 27/30, Loss: 1.0094
Epoch 28/30, Loss: 1.0160
Epoch 29/30, Loss: 1.0278
Epoch 30/30, Loss: 1.0406


In [110]:
idx = 350
text = tokenizer.encode(train_texts[idx])
input_tens = torch.tensor(text, dtype=torch.long).unsqueeze(0).to(device)
out = modelT(input_tens)
max_index_argmax = out.argmax()
print(max_index_argmax)
print(label_map[train_status[idx]])

tensor(1, device='cuda:0')
1
